# Generate .do file

The following script generates a `.do` file to be used on stata in order to translate a .dat file into a .dta file.

In [1]:
# Imports
import sys
import os
sys.path.append('../..')

# Data Manipulation
import pandas as pd
import numpy as np


In [18]:
# Global Variables
DATA_PATH = "../../data/"
LAYOUT_FILE = '../../dados/layouts/2010_Layout_microdados_Amostra_.xlsx'
LOC_FILE = "../../dados/planilhas-api/localidades.csv"

TEST_DATA_FILE = '../data/microdados/amostra_emigracao_2010_AL.txt'

## 1. Read the layout file

In [8]:
# Carrega o arquivo de labels como um arquivo Excel
layout_2010 = pd.ExcelFile(LAYOUT_FILE)

In [9]:
# Funções Auxiliares

def remove_unnamed_columns(df) -> pd.DataFrame:
    """
    Remove colunas com nome Unnamed. Estas colunas vem de colunas que foram mescladas no .xlsx original.

    Args:
        df (pd.DataFrame): DataFrame com colunas Unnamed

    Returns:
        pd.DataFrame: DataFrame sem colunas Unnamed
    
    """
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]


def get_values(split: list):
    values = {}
    for val in range(1, len(split)):
        aux = split[val].split('-')
        if len(aux) == 1:
            aux.append(aux[0])
            aux[0] = ' '    
        # aux[1] = aux[1].replace(' ', '')
        values[aux[0]] = aux[1]
    return values


def split_col_names_values(col: pd.Series): 
    df = pd.DataFrame(columns = ['VAR', 'NOME', 'CHAVE', 'VALOR'])
    for index, value in col.items():
        df_aux = pd.DataFrame(columns = df.columns)
        split = value.split('\n')
        # df_aux['NOME'] = pd.concat([df_aux['NOME'], pd.Series(split[0])])
        if len(split) > 1:
            # series_values = pd.concat([series_values, pd.Series(get_values(split))])
            # df_line = pd.DataFrame(columns = ['NOME', 'CHAVES', 'VALORES'])
            # # df_line['NOME'] = pd.Series(split[0])
            # df_line = pd.Series(get_values)
            # dct = pd.Series(get_values)
            lst = [{'CHAVE': d_key, 'VALOR': d_value, 'VAR': index} for d_key, d_value in get_values(split).items()]
            df_aux = pd.DataFrame(lst)
            df_aux['NOME'] = split[0]
            
        else:
            df_aux['NOME'] = pd.Series(split[0])
            df_aux['VAR'] = index
            df_aux['CHAVE'] = np.nan
            df_aux['VALOR'] = np.nan
        
        df_aux['NOME'] = df_aux['NOME'].str.replace('\n', '')
        df_aux['NOME'] = df_aux['NOME'].str.replace(':', '')
        df = pd.concat([df, df_aux])
    return df

def prepare_df(df: pd.DataFrame)-> tuple:
    df = remove_unnamed_columns(df)
    df_vars = split_col_names_values(df.set_index('VAR')['NOME'])
    return df, df_vars

### 1.X. Testes

In [14]:
# Carrega as labels referentes aos microdados de domicílios
df_domi = pd.read_excel(layout_2010, sheet_name='EMIG')
df_domi, df_domi_vars = prepare_df(df_domi)
df_domi.tail(10)

,VAR,NOME,POSIÇÃO INICIAL,POSIÇÃO FINAL,INT,DEC,TIPO
9,V1006,SITUAÇÃO DO DOMICÍLIO: \n1- Urbana\n2- Rural,53,53,1,NaN,C\n
10,V0303,SEXO DO EMIGRANTE:\n1- Masculino\n2- Feminino\...,54,54,1,NaN,C
11,V0304,ANO DE NASCIMENTO DO EMIGRANTE:\n- Branco\n- 1...,55,58,4,NaN,A
12,V0305,ANO DA ÚLTIMA PARTIDA DO EMIGRANTE:\n- Branco\...,59,62,4,NaN,A
13,V3061,PAÍS DE RESIDÊNCIA EM 31 DE JULHO DE 2010 – CÓ...,63,69,7,NaN,A
14,M0303,MARCA DE IMPUTAÇÃO NA V0303: \n1- Sim\n2- Não,70,70,1,NaN,C\n
15,M0304,MARCA DE IMPUTAÇÃO NA V0304: \n1- Sim\n2- Não,71,71,1,NaN,C
16,M0305,MARCA DE IMPUTAÇÃO NA V0305: \n1- Sim\n2- Não,72,72,1,NaN,C
17,M3061,MARCA DE IMPUTAÇÃO NA V3061: \n1- Sim\n2- Não,73,73,1,NaN,C
18,V1005,Situação do setor \n1 - Área urbanizada\n2 - Á...,74,74,1,NaN,C


In [15]:
df_domi_vars.head()

,VAR,NOME,CHAVE,VALOR
0,V0001,UNIDADE DA FEDERAÇÃO,11,Rondônia
1,V0001,UNIDADE DA FEDERAÇÃO,12,Acre
2,V0001,UNIDADE DA FEDERAÇÃO,13,Amazonas
3,V0001,UNIDADE DA FEDERAÇÃO,14,Roraima
4,V0001,UNIDADE DA FEDERAÇÃO,15,Pará


## 2. Generate .do file

In [16]:
class DoFile:
    """
    Classe para gerar um arquivo .do para o Stata
    """

    def __init__(
            self, 
            file_name: str,
            data_file: str, 
            layout_file: str,
            df_loc_file: str,
            loc_ids: dict = None,
            layout_file_sheet = 0
        ) -> None:
        
        # Levantamento de erros
        if not isinstance(layout_file_sheet, (str, int)):
            raise TypeError("layout_file_sheet deve ser um inteiro ou uma string")

        # Define uma variável para armazenar o conteúdo do arquivo    
        self.file_str = str()
        # Adiciona o cabeçalho do arquivo
        self.add_header()
        # Adiciona o mapeamento das variáveis
        self.add_var_mapping(layout_file, layout_file_sheet)
        # 
        self.add_data_file(data_file)
        # 
        self.add_number_format(layout_file, layout_file_sheet)
        # 
        self.add_labels(layout_file, layout_file_sheet)
        #
        self.add_locations(df_loc_file, loc_ids)
        # Abre o arquivo e escreve o conteúdo
        with open(file_name, 'w', encoding='UTF-8') as file:
            file.write(self.file_str)
            file.close()


    def add_header(
        self
    ) -> None:
        """
        Adiciona o cabeçalho do arquivo .do
        """

        self.file_str += """* NOTE: You need to set the Stata working directory to the path
* where the data file is located.

set more off

clear
        """
        return
    
    def add_var_mapping(
        self, 
        layout_file: str, 
        layout_file_sheet = 0
    ) -> None:
        """Adiciona o mapeamento das variáveis ao .do
        
        Args:
            layout_file (str): Caminho do arquivo de layout
            layout_file_sheet (int, optional): Número da aba do arquivo de layout. Defaults to 0.
        
        """

        self.file_str += """\nquietly infix                ///"""
        # get the layout file
        df = pd.read_excel(layout_file, sheet_name = layout_file_sheet)
        
        # iterate over the df
        for index, row in df.iterrows():

            # get the variable name
            col_name = row['VAR']

            # get the variable length
            col_pos_ini = row['POSIÇÃO INICIAL']
            col_pos_end = row['POSIÇÃO FINAL']

            # get the variable type
            
            # if the column has one position
            if col_pos_ini == col_pos_end:
                # save the variable as a byte
                col_type = 'byte'
            # A represents an identifier and C represents a category
            elif row['TIPO'] == 'A' or row['TIPO'] == 'C':
                # save the id as an int
                col_type = 'int'
            # N represents a numeric variable
            elif row['TIPO'] == 'N':
                # verify if the variable has a decimal part
                if row['DEC'] is None:
                    # if not, save as an int
                    col_type = 'int'
                else:
                    # if yes, save as a double
                    col_type = 'double'

            # write the variable in the file
            self.file_str += f"\n  {col_type}\t\t{col_name}\t\t{col_pos_ini}-{col_pos_end}\t\t///"
                                
    def add_data_file(
        self,
        data_file: str
    ) -> None:
        """
        Adiciona o nome do arquivo de dados ao .do

        Args:
            data_file (str): Caminho do arquivo de dados

        """

        self.file_str += f'\n  using `"{data_file}"\', clear\n'
            

    def add_number_format(self, layout_file, layout_file_sheet = 0):
        """
        Adiciona o formato numérico das variáveis ao .do
        """
        df = pd.read_excel(layout_file, sheet_name = layout_file_sheet)
        # df = split_col_names_values(df.set_index('VAR')['NOME'])
        df['DEC'] = df['DEC'].fillna(0)
        
        number_format_str = str()
        for index, row in df.iterrows():
            if row['TIPO'] == 'N':
                if row['DEC'] == 0:
                    format = f'%0{row["INT"]}.0f'
                else:
                    format = f'%0{int(row["INT"]) + int(row["DEC"]) + 1}.{int(row["DEC"])}f'
                number_format_str += f'\nformat {row["VAR"]} {format}'
        self.file_str += number_format_str + '\n'


    def add_labels(
        self,
        layout_file: str, 
        layout_file_sheet = 0
    ) -> None:
        """
        Adiciona os labels ao .do

        Args:
            layout_file (str): Caminho do arquivo de layout
            layout_file_sheet (int, optional): Número da aba do arquivo de layout. Defaults to 0.

        """

        df = pd.read_excel(layout_file, sheet_name = layout_file_sheet)
        df = split_col_names_values(df.set_index('VAR')['NOME'])
        
        label_var_str = str()
        label_define_str = str()
        label_values_str = str()

        for index, row in df.iterrows():
            # adiciona o nome da variável e possíveis valores à string principal
            label_var_str += f'\nlabel var {row["VAR"]}\t\t`\"{row["NOME"]}\"\''
            if row['CHAVE'] == ' ':
                row['CHAVE'] = '.a'
            if row['CHAVE'] is not np.nan:
                label_define_str += f'\nlabel define {row["VAR"]}_lbl {row["CHAVE"]} `\"{row["VALOR"]}\"\', add'
                label_values_str += f'\nlabel values {row["VAR"]} {row["VAR"]}_lbl'

        # Remove valores repetidos
        label_var_str = '\n'.join(set(label_var_str.split('\n')))
        label_values_str = '\n'.join(set(label_values_str.split('\n')))
        # Adiciona os labels à string principal do arquivo .do
        self.file_str += label_var_str + '\n'
        self.file_str += label_define_str + '\n'
        self.file_str += label_values_str + '\n'
    
    def add_locations(self, df_loc_file, loc_ids: dict = None):
        """
        Adiciona os labels de localização ao .do
        """
        # Define os nomes das colunas com id e nome de cada localidade
        df_loc = pd.read_csv(df_loc_file)

        if loc_ids is None:
            loc_ids = {
                'V0002': ('municipio_id', 'municipio_nome'),
                'V1002': ('mesorregiao_id', 'mesorregiao_nome'),
                'V1003': ('microrregiao_id', 'microrregiao_nome'),
            }

        location_str = str()
        label_values_str = str()
        generate_str = str()
        replace_drop_str = str()

        generate_str = "gen V0001_str = string(V0001)\n"
        
        for key, value in loc_ids.items():
            # extrai do dataset todos os pares únicos de id e nome
            df_aux = df_loc[[value[0], value[1]]].drop_duplicates()
            # itera sobre o dataset
            for index, row in df_aux.iterrows():
                # adiciona os labels à string principal
                location_str += f'\nlabel define {key}_lbl {str(row[value[0]])} `\"{row[value[1]]}\"\', add'
            # gera variável concatenando V0001 e as variáveis de localização
            generate_str += f"""
format {key} %0{len(str(row[value[0]])) - 2}.0f

gen {key}_str = string({key}, "%0{len(str(row[value[0]])) - 2}.0f")
gen {key}_new = V0001_str + {key}_str
destring {key}_new, replace
            """
            # adiciona os labels à string principal
            label_values_str += f'\nlabel values {key} {key}_lbl'
            # adiciona a variável ao replace
            replace_drop_str += f'\nreplace {key} = {key}_new\ndrop {key}_new {key}_str\n'

        replace_drop_str = '\ndrop V0001_str\n' + replace_drop_str        
        # Concatena com a string principal do arquivo .do        
        self.file_str += location_str + '\n' + generate_str + replace_drop_str + '\n' +  label_values_str + '\n'

In [20]:
# domi_file = pd.read_excel(layout_2010, sheet_name='DOMI')

dofile = DoFile('test.do', TEST_DATA_FILE, layout_2010, LOC_FILE ,layout_file_sheet='EMIG')